# Sweeps - Eigenmode matrix

### Prerequisite
You need to have a working local installation of Ansys

## 1. Perform the necessary imports and create a QDesign in Metal first.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
from qiskit_metal.analyses.quantization import EPRanalysis

In [3]:
# Create the design in Metal
# Create a design by specifying the chip size and open Metal GUI.

design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'

gui = MetalGUI(design)

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

### In this example, the design consists of 1 qubit and 1 CPW connected to OpenToGround.

In [4]:
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

# Remove all qcomponents from GUI.
design.delete_all_components()

# So as to demonstrate the quality factor outputs easily, the
#subtrate material type is being changed to FR4_epoxy from the
#default of silicon
design.chips.main.material = 'FR4_epoxy'

q1 = TransmonPocket(
    design,
    'Q1',
    options=dict(pad_width='425 um',
                 pocket_height='650um', 
                 hfss_inductance = '17nH',
                 connection_pads=dict(
                     readout=dict(loc_W=+1, loc_H=+1, pad_width='200um'))))
otg = OpenToGround(design,
                   'open_to_ground',
                   options=dict(pos_x='1.75mm', pos_y='0um', orientation='0'))
readout = RouteMeander(
    design, 'readout',
    Dict(
        total_length='6 mm',
        hfss_wire_bonds = True,
        fillet='90 um',
        lead=dict(start_straight='100um'),
        pin_inputs=Dict(start_pin=Dict(component='Q1', pin='readout'),
                        end_pin=Dict(component='open_to_ground', pin='open')),
    ))

gui.rebuild()
gui.autoscale()

## 2 Metal passes information to 'hfss' simulator, and gets a solution matrix.


In [5]:
# Create a separate analysis object for the combined qbit+readout.
eig_qres = EPRanalysis(design, "hfss")


Prepare data to pass as arguments for method run_sweep().  

Method run_sweep() will open the simulation software if software is not open already.

In [6]:
### for render_design()
# Render every QComponent in QDesign.
render_qcomps = []

# Identify which kind of pins in Ansys. 
# Follow details from renderer in
# QHFSSRenderer.render_design.
# No pins are open, so don't need to utilize render_endcaps.
open_terminations = []

#List of tuples of jj's that shouldn't be rendered.  
#Follow details from renderer in QHFSSRenderer.render_design.
render_ignored_jjs = []

# Either calculate a bounding box based on the location of 
# rendered geometries or use chip size from design class.
box_plus_buffer = True

In [7]:
# For simulator hfss, the setup options are :
# min_freq_ghz, n_modes, max_delta_f, max_passes, min_passes, min_converged=None,
# pct_refinement, basis_order

# If you don't pass all the arguments, the default is determined by 
# QHFSSRenderer's default_options.

# If a setup named "sweeper_em_setup" exists in the project, it will be deleted, 
# and a new setup will be added.

eig_qres.sim.setup.name="sweeper_em_setup"
eig_qres.sim.setup.min_freq_ghz=4
eig_qres.sim.setup.n_modes=2
eig_qres.sim.setup.max_passes=15
eig_qres.sim.setup.min_converged = 2
eig_qres.sim.setup.max_delta_f = 0.2

eig_qres.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj'
eig_qres.setup.junctions.jj.line = 'JJ_Lj_Q1_rect_jj_'


### - Connect to Ansys HFSS, eigenmode solution.
### - Rebuild QComponents in Metal.
### - Render QComponents within HFSS and setup.
### - Delete/Clear the HFSS between each calculation of solution matrix.
### - Calculate solution matrix for each value in option_sweep.

#### Return a dict and return code.  If the return code is zero, there were no errors detected.  
#### The dict has:  key = each value used to sweep, value = data from simulators

#### This could take minutes based size of design.

In [8]:
#Note: The method will connect to Ansys, activate_eigenmode_design(), add_eigenmode_setup().

all_sweeps, return_code = eig_qres.run_sweep(readout.name,
                                        'total_length', 
                                        ['10mm', '11mm', '12mm'],
                                        render_qcomps,
                                        open_terminations,
                                         ignored_jjs=render_ignored_jjs,
                                        design_name="GetEigenModeSolution", 
                                       box_plus_buffer=box_plus_buffer
                                      )


INFO 02:32PM [connect_project]: Connecting to Ansys Desktop API...
INFO 02:32PM [load_ansys_project]: 	Opened Ansys App
INFO 02:32PM [load_ansys_project]: 	Opened Ansys Desktop v2023.1.0
INFO 02:32PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/askev/OneDrive/文件/Ansoft/
	Project:   Project3
INFO 02:32PM [connect_design]: 	Opened active design
	Design:    HangerReso_hfss [Solution type: Eigenmode]
INFO 02:32PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 02:32PM [connect]: 	Connected to project "Project3" and design "HangerReso_hfss" 😀 

INFO 02:32PM [connect_design]: 	Opened active design
	Design:    GetEigenModeSolution_hfss [Solution type: Eigenmode]
WARNING 02:32PM [connect_setup]: 	No design setup detected.
WARNING 02:32PM [connect_setup]: 	Creating eigenmode default setup.
INFO 02:32PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 02:33PM [get_setup]: 	Opened setup `sweeper_em_setup`  (<class 'pyEPR

Design "GetEigenModeSolution_hfss" info:
	# eigenmodes    2
	# variations    1
Design "GetEigenModeSolution_hfss" info:
	# eigenmodes    2
	# variations    1


  options=pd.Series(get_instance_vars(self.options)),




        energy_elec_all       = 4.46055685058983e-25
        energy_elec_substrate = 3.62915006868395e-25
        EPR of substrate = 81.4%

        energy_mag    = 6.90020898166949e-27
        energy_mag % of energy_elec_all  = 1.5%
        

Variation 0  [1/1]


  Ljs = pd.Series({})

  Cjs = pd.Series({})

  _Om = pd.Series({})



...

  Qp = pd.Series({})



		Energy fraction (Lj over Lj&Cj)= 95.62%
	jj               1.34101  (+)        0.0614871
		(U_tot_cap-U_tot_ind)/mean=-18.75%
Calculating Qdielectric_main for mode 1 (1/1)


  sol = sol.append(self.get_Qdielectric(



p_dielectric_main_1 = 0.813275110797672


03:36PM 13s WARNING [iterate_option_sweep]: For class EPRanalysis, run() did not execute as expected: An exception of type com_error occurred. Arguments:
(-2147352567, '發生例外狀況。', (0, None, None, None, 0, -2147024382), None)


In [9]:
all_sweeps.keys()

dict_keys(['10mm', '11mm', '12mm'])

In [10]:
# For example, just one group of solution data.
all_sweeps['10mm'].keys()


dict_keys(['option_name', 'variables', 'sim_variables'])

In [11]:
all_sweeps['10mm']


{'option_name': 'total_length',
 'variables': {'energy_elec': 4.46055685058983e-25,
  'energy_elec_sub': 3.62915006868395e-25,
  'energy_mag': 6.90020898166949e-27},
 'sim_variables': {'sim_setup_name': 'sweeper_em_setup',
  'convergence_t':              Solved Elements  Max Delta Freq. %
  Pass Number                                    
  1                      12901                NaN
  2                      16773           22.64400
  3                      21809            8.93140
  4                      27824            6.06480
  5                      34773            3.33890
  6                      45211            2.10040
  7                      58789            1.63620
  8                      76432            1.04290
  9                      99364            0.59636
  10                    129178            0.33703
  11                    167931            0.33750
  12                    218311            0.30980
  13                    283795            0.45558
  14      

In [12]:
all_sweeps['10mm']['variables']

{'energy_elec': 4.46055685058983e-25,
 'energy_elec_sub': 3.62915006868395e-25,
 'energy_mag': 6.90020898166949e-27}

In [13]:
all_sweeps['10mm']['sim_variables']['convergence_t']

,Solved Elements,Max Delta Freq. %
Pass Number,,
1,12901,NaN
2,16773,22.64400
3,21809,8.93140
4,27824,6.06480
5,34773,3.33890
6,45211,2.10040
7,58789,1.63620
8,76432,1.04290
9,99364,0.59636


In [14]:
all_sweeps['10mm']['sim_variables']['convergence_f']

,re(Mode(1)) [g],re(Mode(2)) [g]
Pass [],,
1,4.988856,6.339131
2,6.118529,7.161170
3,6.665001,7.586173
4,7.000362,8.046258
5,7.173020,8.314917
6,7.279428,8.489565
7,7.335418,8.628472
8,7.373493,8.718458
9,7.401210,8.770451


In [15]:
# Uncomment the next close simulation software. 
#eig_qres.sim.close()

In [16]:
# Uncomment next line if you would like to close the gui
#gui.main_window.close()